In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
# read in initial data and drop columns not included in the model
cols_to_drop = ['play_id', 'game_id', 'yards_gained', 'qb_scramble', 'qb_dropback', 'kick_distance', 'punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty', 'third_down_converted', 'third_down_failed', 'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'safety', 'penalty', 'fumble_lost', 'qb_hit', 'sack', 'touchdown', 'extra_point_attempt', 'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'passing_yards', 'passer_player_name', 'receiving_yards', 'receiver_player_id', 'receiver_player_name', 'rushing_yards', 'rusher_player_id', 'rusher_player_name', 'penalty_team', 'replay_or_challenge', 'replay_or_challenge_result', 'weather']
df = pd.read_csv('../cleaned_data.csv')
df = df.drop(columns=cols_to_drop)

C:\Users\HENSLEYAS21\AppData\Local\Temp\ipykernel_7412\3823286429.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../cleaned_data.csv')


In [17]:
one_hot_encoding_fields = [
    "home_team",
    "away_team",
    "posteam",
    "defteam",
    "game_half",
    "passer_player_id",
    "stadium",
    "roof",
    "surface",
    "home_coach",
    "away_coach"
]
df = pd.get_dummies(df, columns=one_hot_encoding_fields)
print(f'num cols = {len(df.columns)}')

num cols = 1383


In [18]:
# make prediction using baseline model

# baseline model:
#   if down = 1:    run
#   if down = 2:    pass
#   if down = 3:    pass
#   if down = 4 and yardline_100 > 30:     punt
#   if down = 4 and yardline_100 <= 30:    field_goal

down_to_play = {1:'run', 2:'pass', 3:'pass', 4:'punt'}
df['play_prediction'] = df['down'].map(down_to_play)
df.loc[(df['down'] == 4) & (df['yardline_100'] <= 30), 'play_prediction'] = 'field_goal'

In [ ]:
num_correct_df = df.loc[df['play_prediction'] == df['play_type']]
print(len(num_correct_df) / len(df))

0.6053285274293622


In [19]:
X = df.drop(columns=['play_type'])
y = df['play_type']
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

672090
224030
672090
224030
